In [1]:
%pip install datasets transformers onnx onnxruntime

In [2]:

model_name = 'microsoft/xtremedistil-l6-h256-uncased'


from datasets import load_dataset
dataset = load_dataset("emotion")
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)
tokenized_datasets = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [3]:


full_train_dataset = tokenized_datasets["train"]
full_eval_dataset = tokenized_datasets["test"]


In [4]:
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)


cpu


In [5]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=6)
model = model.to(device)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/xtremedistil-l6-h256-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

<ipython-input-6-4f83f333ec09>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [7]:
pip install transformers[torch]

In [8]:
pip install accelerate -U

In [9]:
from transformers import TrainingArguments
training_args = TrainingArguments("test_trainer",
                                  per_device_train_batch_size=128,
                                  num_train_epochs=4,learning_rate=3e-05,
                                  evaluation_strategy="epoch")
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=full_train_dataset,
    eval_dataset=full_eval_dataset,
    compute_metrics=compute_metrics,
)

In [10]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.279429,0.610000
2,No log,1.069831,0.637500


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.279429,0.610000
2,No log,1.069831,0.637500
3,No log,0.962878,0.695500
4,1.199100,0.929085,0.704500


TrainOutput(global_step=500, training_loss=1.1990584716796875, metrics={'train_runtime': 6808.5301, 'train_samples_per_second': 9.4, 'train_steps_per_second': 0.073, 'total_flos': 236244467712000.0, 'train_loss': 1.1990584716796875, 'epoch': 4.0})

In [13]:
trainer.evaluate()


{'eval_loss': 0.9290845394134521,
 'eval_accuracy': 0.7045,
 'eval_runtime': 83.018,
 'eval_samples_per_second': 24.091,
 'eval_steps_per_second': 3.011,
 'epoch': 4.0}

In [14]:
import transformers
import transformers.convert_graph_to_onnx as onnx_convert
from pathlib import Path

In [15]:
pipeline = transformers.pipeline("text-classification",model=model,tokenizer=tokenizer)

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [16]:
model = model.to("cpu")

In [17]:
onnx_convert.convert_pytorch(pipeline, opset=11, output=Path("classifier.onnx"), use_external_format=False)

Using framework PyTorch: 2.0.1+cu118
Found input input_ids with shape: {0: 'batch', 1: 'sequence'}
Found input token_type_ids with shape: {0: 'batch', 1: 'sequence'}
Found input attention_mask with shape: {0: 'batch', 1: 'sequence'}
Found output output_0 with shape: {0: 'batch'}
Ensuring inputs are in correct order
position_ids is not present in the generated input list.
Generated inputs order: ['input_ids', 'attention_mask', 'token_type_ids']
============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [ ]:
pip install optimum-transformers